In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np


In [2]:

# Load and Preprocess Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    classes=['Red', 'Trout', 'Flounder']  # List the class names explicitly
)

Found 423 images belonging to 3 classes.


In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/Users/albertoalvarado/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.fit(train_generator, epochs=50, batch_size=32)


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 245ms/step - accuracy: 0.6053 - loss: 0.8529
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 281ms/step - accuracy: 0.5399 - loss: 0.9069
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 281ms/step - accuracy: 0.5962 - loss: 0.8877
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 248ms/step - accuracy: 0.5823 - loss: 0.8860
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.5628 - loss: 0.9078
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 242ms/step - accuracy: 0.6473 - loss: 0.8460
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 270ms/step - accuracy: 0.6334 - loss: 0.8325
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 278ms/step - accuracy: 0.6319 - loss: 0.8437
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 248ms/step - accuracy: 0.6280 - loss: 0.7872
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 272ms/step - accuracy: 0.6237 - loss: 0.8212
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 275ms/step - accuracy: 0.6314 - loss: 0.7635
Epoch 12/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 269ms/step

In [11]:
model.save('fish_identifier_model.h5')


In [10]:
# Prediction Function
def predict_fish(image_path, model):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    
    # Define the class labels based on your classes
    class_labels = ['Red', 'Trout', 'Flounder']
    
    predicted_class = class_labels[predicted_class_index]
    
    return predicted_class

# Example usage:
image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/Project4/FishExamples/testtrout.png'
predicted_fish_class = predict_fish(image_path, model)
print("Predicted fish class:", predicted_fish_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted fish class: Trout


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_fish(image_path, model):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    
    # Get the class labels from the generator
    class_labels = list(train_generator.class_indices.keys())
    
    predicted_class = class_labels[predicted_class_index]
    
    return predicted_class

# Example usage:
image_path = 'https://gulfcoastmariner.s3.amazonaws.com/2017/09/squarebill-trout.jpg'
predicted_fish_class = predict_fish(image_path, model)
print("Predicted fish class:", predicted_fish_class)


In [9]:
import requests
from io import BytesIO

def predict_fish_from_url(image_url, model):
    response = requests.get(image_url)
    img = image.load_img(BytesIO(response.content), target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    
    # Assuming you have access to the class labels
    class_labels = ['Red', 'Trout', 'Flounder']
    
    predicted_class = class_labels[predicted_class_index]
    
    return predicted_class

# Example usage:
image_url = 'https://www.usm.edu/ocean-science-engineering/images/red-drum.jpg'
predicted_fish_class = predict_fish_from_url(image_url, model)
print("Predicted fish class:", predicted_fish_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted fish class: Red
